In [7]:
import mediapipe as mp
import numpy as np
import torch
#import torch_geometric
from PIL import Image

from torch_geometric.data import Data
import cv2
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import glob
import os
import math 
from IPython.display import clear_output
import itertools
import seaborn as sb
from tqdm import tqdm
device = 

In [8]:
import torch
from transformers import ViTModel, ViTConfig
from torch import nn

# بارگذاری مدل ViT با پیکربندی پایه
config = ViTConfig.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k", config=config)

# استفاده از تنها سه لایه اول
class ViTThreeLayer(nn.Module):
    def __init__(self, vit_model):
        super(ViTThreeLayer, self).__init__()
        # ذخیره تنها سه لایه اول
        self.layer1 = vit_model.encoder.layer[:3]
    
    def forward(self, x):
        # عبور از لایه ورودی (مانند کد نویسی ها و توکن بندی)
        outputs = model.embeddings(x)
        
        # عبور از سه لایه اول
        for layer in self.layer1:
            outputs = layer(outputs)[0]  # گرفتن تنها آخرین وضعیت مخفی
        
        # استخراج ویژگی‌ها برای توکن [CLS] و فلت کردن آن به یک بردار یک‌بعدی
        cls_token = outputs[:, 0, :]
        return cls_token.view(-1)

# نمونه‌سازی از مدل با سه لایه
vit_three_layer = ViTThreeLayer(model)

# ورودی تصادفی برای تست (1 تصویر با اندازه 224x224 و 3 کانال)
dummy_input = torch.randn(1, 3, 224, 224)

# اجرای مدل و دریافت خروجی
with torch.no_grad():
    output = vit_three_layer(dummy_input)

print("Output shape:", output.shape)


Output shape: torch.Size([768])


In [9]:
kernel_size= 20
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh

connections = mp_face_mesh.FACEMESH_TESSELATION
edges = []
for connection in connections:
    edge = [connection[0], connection[1]]
    edges.append(edge)
    edge  =[connection[1], connection[0]]
    edges.append(edge)
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [10]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/oulu/preposess/*/cropped*.jpg")

In [ ]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        torch.from_numpy(patches).to(device)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/'+os.path.basename(path)+'.pt')
